In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas


spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

#filtered_df = df.filter(df.installation_id is unique)
# display(df.select("installation_id").distinct())
# display(df.select("tag").distinct())

df = spark.sql("""
               select distinct installation_id
               from test_unify_analytics.bronze.logs_partition_1
              """)
list = df.toPandas()['installation_id'].tolist()
print(list)

# df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
#         .select('installation_id', 'tag', 'unix_timestamp', 'values') \
#         .filter((col("tag") == 242) & (col('values')[5] == 14))
# df = spark.sql("""
#                 select installation_id, tag, unix_timestamp, values
#                 from test_unify_analytics.bronze.logs_partition_1
#                 where tag = 242 AND values[5] = 14
#                 """)
# display(df)




In [0]:
%sql
select installation_id, tag, count(tag)
from test_unify_analytics.bronze.logs_1
where installation_id = 'a037a00000pwUFKAA2' and (tag = 248 or tag = 249)
group by installation_id, tag
order by tag asc;

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

ps_df = spark.table('test_unify_analytics.bronze.logs_partition_1')
ps_df = ps_df.filter((ps_df.installation_id == 'a030Y00000KPfzuQAD') & (ps_df.tag == 242))

# ps_df1 = ps_df.select(ps_df.installation_id, ps_df.tag, ps_df.unix_timestamp, ps_df.values[1].alias('bin num'))
# pd_df1 = ps_df1.toPandas()
# pd_df1 = pd_df1.sort_values(by=['bin num', 'unix_timestamp'])
# display(pd_df1)
# print('num rows: ', pd_df1.size)

ps_df2 = ps_df.select(ps_df.values[0].alias('bin num'), ps_df.values[5].alias('mode')).filter(ps_df.values[5] == 14)
ps_df2 = ps_df2.groupBy(['bin num', 'mode']).count()

pd_df2 = ps_df2.toPandas().sort_values(by=['count'], ascending=False)
display(pd_df2)

In [0]:
import pandas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Installation Data").getOrCreate()
        
# for i in range (0,4):
# values[5] == 14 --> bin in PORT
df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
       .select('installation_id', 'tag', 'unix_timestamp', 'values') \
       .filter((col('installation_id') == 'a031v00001GpVAEAA3') & 
               (col('tag') == 242) & 
               (col('values')[5] == 14))


df1 = df.withColumn('bin num', col('values')[0]) \
       .withColumn('unix timestamp', col('unix_timestamp'))
window_spec = Window.partitionBy('bin num').orderBy('unix_timestamp')
df1 = df1.withColumn('prev_timestamp', lag('unix_timestamp').over(window_spec)) \
       .withColumn('time diff (sec)', col('unix_timestamp') - col('prev_timestamp'))

result_df = df1.groupBy('bin num').agg(round(mean('time diff (sec)')).alias('avg time between presentations (sec)'))
result_df = result_df.withColumn('avg time between presentations (min)', round(col('avg time between presentations (sec)') / 60))
pd = result_df.toPandas().sort_values(by=['avg time between presentations (sec)'])
display(pd)

# pd.to_csv('/dbfs/FileStore/tables/a031v00001GpVAEAA3.csv')
# # retrieve_filepath = '/files/tables/a031v00001GpVAEAA3.csv'
# url = 'https://adb-4786482868192795.15.azuredatabricks.net/files/tables/a031v00001GpVAEAA3.csv'
# response = requests.get(url)
# print(response)

In [0]:
# import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import webbrowser

spark = SparkSession.builder.appName("Installation Data").getOrCreate()

install_df = spark.sql("""
              select distinct installation_id
              from test_unify_analytics.bronze.logs_partition_1
              """)
installation_list = install_df.toPandas()['installation_id'].tolist()

df = spark.sql("""
              select installation_id, tag, unix_timestamp, values
              from test_unify_analytics.bronze.logs_partition_1
              where tag = 242 AND values[5] = 14
              """)

# df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
#         .select('installation_id', 'tag', 'unix_timestamp', 'values') \
#         .filter((col("tag") == 242) & (col('values')[5] == 14)) \
#         # .cache()

# install_df = df.select('installation_id').distinct()
# installation_list = [row['installation_id'] for row in install_df.toLocalIterator()]
# print(len(installation_list))

# df.cache()
dbutils.fs.mkdirs('dbfs:/FileStore/tables')

# values[5] == 14 --> bin in PORT
for installation in installation_list:
       df_loop = df.filter((col('installation_id') == installation)) 

       df_loop = df_loop.withColumn('bin num', col('values')[0]) \
                        .withColumn('unix timestamp', col('unix_timestamp'))
       window_spec = Window.partitionBy('bin num').orderBy('unix_timestamp')
       df_loop = df_loop.withColumn('prev_timestamp', lag('unix_timestamp').over(window_spec)) \
                        .withColumn('time diff (sec)', col('unix_timestamp') - col('prev_timestamp'))

       result_df = df_loop.groupBy('bin num').agg(round(mean('time diff (sec)')).alias('avg time between presentations (sec)'))
       result_df = result_df.withColumn('avg time between presentations (min)', round(col('avg time between presentations (sec)') / 60))
              
       pd = result_df.toPandas().sort_values(by=['avg time between presentations (sec)'])
       # display(pd)

       store_filepath = '/dbfs/FileStore/tables/' + installation + '.csv'
       pd.to_csv(store_filepath)

       #  retrieve_filepath = '/files/tables/' + installation + '.csv'
       #  url = 'https://adb-4786482868192795.15.azuredatabricks.net' + retrieve_filepath
       #  webbrowser.open(url)

In [0]:
import pandas as pd

# dbutils.fs.rm('/FileStore/tables/', recurse=True)
# dbutils.fs.mkdirs('dbfs:/FileStore/tables')
files = dbutils.fs.ls('/FileStore/tables/')
display(files)
# l = [f.name for f in files]
# print('num files: ', len(l))
# display(pd.DataFrame(l, columns=['installations']))

## Prepare all DataFrames that will be parsed through later 

**install_df**: get all unique installation_id to create installation_list that will later be iterated though to get relevant data for all installations \
**df**: get all BIN_UPDATE ASLog entries where bin mode is PORT \
**timestamp_df**: get local timestamp for each BIN_UPDATE where bin mode is PORT and convert that timestamp to a day of the week

In [0]:
import pandas
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Installation Data").getOrCreate()

install_df = spark.sql("""
                select distinct installation_id
                from test_unify_analytics.bronze.logs_partition_1
            """)
# display(install_df)
installation_list = install_df.toPandas()['installation_id'].tolist()
# print(len(installation_list))

# values[5] == 14 --> bin in PORT
df = spark.sql("""
            select installation_id, tag, unix_timestamp, values
            from test_unify_analytics.bronze.logs_partition_1
            where tag = 242 AND values[5] = 14
        """)

# values[5] == 14 --> bin in PORT
timestamp_df = spark.sql("""
                    select local_installation_timestamp, values[0] as BinNo
                    from test_unify_analytics.bronze.logs_partition_1
                    where tag = 242 AND values[5] = 14
                """)
timestamp_df = timestamp_df.withColumn('dotw', date_format(col('local_installation_timestamp'), 'EEEE'))

In [0]:
import pandas
from pyspark.sql.functions import *
from pyspark.sql.window import Window

dbutils.fs.mkdirs('dbfs:/FileStore/tables/avg_bin_time')
for installation in installation_list:
    df_loop = df.filter((col('installation_id') == installation)) 

    df_loop = df_loop.withColumn('bin num', col('values')[0]) \
                     .withColumn('unix timestamp', col('unix_timestamp'))
    window_spec = Window.partitionBy('bin num').orderBy('unix_timestamp')
    df_loop = df_loop.withColumn('prev_timestamp', lag('unix_timestamp').over(window_spec)) \
                     .withColumn('time diff (sec)', col('unix_timestamp') - col('prev_timestamp'))

    result_df = df_loop.groupBy('bin num').agg(round(mean('time diff (sec)')).alias('avg time between presentations (sec)'))
    result_df = result_df.withColumn('avg time between presentations (min)', round(col('avg time between presentations (sec)') / 60))
            
    pd = result_df.toPandas().sort_values(by=['avg time between presentations (sec)'])
    # display(pd)

    store_filepath = '/dbfs/FileStore/tables/avg_bin_time/' + installation + '.csv'
    pd.to_csv(store_filepath)

In [0]:
import pandas as pd
from datetime import *
from pyspark.sql.functions import *

dbutils.fs.mkdirs('dbfs:/FileStore/tables/dotw')
for installation in installation_list:
    df_dotw = timestamp_df.filter((col('installation_id') == installation)).groupBy(['BinNo', 'dotw']).count()
    # timestamp_df = timestamp_df.groupBy(['BinNo', 'dotw']).count()
    pd_dotw = df_dotw.orderBy(desc("BinNo"), desc("count")).toPandas()

    store_filepath = '/dbfs/FileStore/tables/dotw/' + installation + '.csv'
    pd_dotw.to_csv(store_filepath)

# time_list = timestamp_df.toPandas()['local_installation_timestamp'].tolist()
# print(time_list[0].day_name())
# for i, val in enumerate(time_list):
#     time_list[i] = val.to_pydatetime()
# print(time_list)

# dotw = time_list[0].strftime("%A")
# print(dotw)